# Práctica: Los Juegos del Hambre

ESTUDIANTE: `ADRIÁN FERNÁNDEZ UGUINA`

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

En esta práctica vamos a enfrentarnos a un problema desafiante de clasificación de imágenes, construyendo una red neuronal profunda que sea capaz de clasificar entre diferentes tipos de comida. ¡Que comiencen los Juegos del Hambre!

## Guidelines

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema, y a conseguir una calificación más alta. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb), y de haber [deactivado otras sesiones que tuvieras activas](https://stackoverflow.com/a/53441194/2436578).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Evita mensajes de advertencia de TensorFlow
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Índice de la GPU a utilizar (0, 1, 2, etc.)

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Obtención de datos

Vamos a usar un dataset de imágenes de comida disponible en [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). Para descargarlo, necesitarás crear una cuenta de usuario en Kaggle, y obtener tus credenciales de la API. Puedes hacerlo siguiendo las instrucciones de [esta sección](https://github.com/Kaggle/kaggle-api#api-credentials). ¡Ojo! Tus credenciales de la API no son lo mismo que la contraseña que utilizas para acceder a tu cuenta en Kaggle.

Una vez tengas el fichero JSON con tus credenciales, puedes declararlas en este notebook asignando las variables de entorno adecuadas, de la siguiente manera

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Cuando lo hayas hecho, podrás descargar el dataset a la máquina donde esté corriendo este notebook usando el siguiente comando

    
    
    
donde debes indicar el nombre de un directorio válido como "YOUR_LOCAL_FOLDER". Si prefieres descargar los datos en la misma carpeta que este notebook, puedes quitar la parte `-p YOUR_LOCAL_FOLDER` del comando.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Crea tu propia cuenta de Kaggle (si no tienes ya una), obtén tus credenciales, y usa la celda inferior para declarar tu nombre de usuario de Kaggle y tu clave como variables de entorno. A continuación, usa la misma celda para descargar el dataset de imágenes.
    
¡Ojo! Debes mantener estas credenciales en secreto, ya que son personales a tu usuario de Kaggle. Recuerda borrarlas de la celda antes de entregar este notebook.
</font>

***

In [6]:
####### INSERT YOUR CODE HERE


os.environ["KAGGLE_USERNAME"] = "adrinfernndezuguina"
os.environ["KAGGLE_KEY"] = "ade9590da40cdfa403674d914d8dd141"

!kaggle datasets download trolukovich/food11-image-dataset --unzip 

 98% 1.07G/1.08G [00:09<00:00, 211MB/s]
100% 1.08G/1.08G [00:09<00:00, 118MB/s]


Revisa ahora la carpeta en la que has descargado los datos. Verás que contiene 3 subdirectorios:

* **training**, contiene las imágenes a utilizar para entrenar el modelo.
* **validation**, contiene imágenes adicionales que podrías usar como datos de entrenamiento adicionales, o para algún tipo de estrategia de validación como Early Stopping.
* **evaluation**, contiene las imágenes que debes utilizar para testear el modelo. Las imágenes de esta carpeta **solo** pueden utilizarse para medir el rendimiento del modelo tras su entrenamiento, y para nada más.

Además de esto, dentro de cada una de estas carpetas encontrarás una subcarpeta para cada una de las 11 clases de comida:

* Bread (panes)
* Dairy product (lácteos)
* Dessert (postres)
* Egg (huevos)
* Fried food (fritos)
* Meat (carnes)
* Noodles-Pasta (pasta)
* Rice (arroz)
* Seafood (pescado y marisco)
* Soup (sopas)
* Vegetable-Fruit (vegetales y frutas)

Esta es una forma estándar de organizar los datasets de imágenes: una carpeta para cada clase. Para facilitar los pasos de procesamiento que vendrán a continuación, vamos a definir algunas variables que nos indiquen dónde están almacenados los diferentes conjuntos de datos.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea variables <b>TRAINDIR</b>, <b>VALDIR</b> y <b>TESTDIR</b>, cada una conteniendo una cadena de texto con la ruta al directorio donde están los datos de entrenamiento, validación y evaluación, respectivamente.
</font>

***

In [7]:
####### INSERT YOUR CODE HERE
TRAINDIR = "/content//training"
VALDIR = "/content//validation"
TESTDIR = "/content//evaluation"

if not os.path.isdir(TRAINDIR):
    print(f"El directorio de entrenamiento '{TRAINDIR}' no existe.")
if not os.path.isdir(VALDIR):
    print(f"El directorio de validación '{VALDIR}' no existe.")                                                                                                     
if not os.path.isdir(TESTDIR):
    print(f"El directorio de test '{TESTDIR}' no existe.")

### Reducción de clases

Con el fin de hacer este problema más accesible de cara a la práctica, vamos a centrarnos solo en seis de las clases de comida disponibles: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` y `Meat`. Para ello, se provee el código siguiente, que elimina de los datos descargados las carpetas correspondientes a imágenes de las otras clases.

In [8]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder




Deleting /content//evaluation/Vegetable-Fruit...
Deleting /content//evaluation/Noodles-Pasta...
Deleting /content//evaluation/Soup...
Deleting /content//evaluation/Seafood...
Deleting /content//evaluation/Rice...
Deleting /content//training/Vegetable-Fruit...
Deleting /content//training/Noodles-Pasta...
Deleting /content//training/Soup...
Deleting /content//training/Seafood...
Deleting /content//training/Rice...
Deleting /content//validation/Vegetable-Fruit...
Deleting /content//validation/Noodles-Pasta...
Deleting /content//validation/Soup...
Deleting /content//validation/Seafood...
Deleting /content//validation/Rice...


## Procesando imágenes desde ficheros

Este dataset de imágenes es grande, con imágenes de mayor resolución que las que hemos utilizado en el tutorial del MNIST, y cada una de ellas teniendo diferentes tamaños y relación de aspecto. Además, mientras que para el MNIST teníamos una función de keras que preparaba los datos para nosotros, en esta ocasión tendremos que realizar el trabajo de carga y procesamiento de las imágenes.

Una forma conveniente de hacer todo este trabajo es a través de la función Keras `image_dataset_from_directory`. Esta función crea un objeto `Dataset` de TensorFlow con todas las imágenes de un directorio, cargándolas en memoria de forma dinámica solo cuando la red neuronal necesita utilizarlas. Esta función también nos permite especificar algunas opciones de preprocesamiento muy útiles.

Por ejemplo, podemos crear un `Dataset` con los datos en la carpeta de training:

In [9]:
from tensorflow.keras.preprocessing import image_dataset_from_directory


image_size = 128
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Found 6082 files belonging to 6 classes.


Observa los parámetros que se han utilizado para configurar el dataset:

* El **directorio** desde el que cargar las imágenes.
* Un **tamaño de imagen (image_size)** que se utilizará para redimensionar todas las imágenes cargadas a ese tamaño común, en este caso 32x32 píxeles.
* El **tamaños de los lotes (batch_size)** de imágenes a ser generados. Nótese que definimos aquí este parametro en lugar de en el paso `fit` de la red, como hemos hecho en otros ejercicios, porque el objeto `Dataset` resultante hará uso de esta información para mantener en memoria solo algunos batches de imágenes, ahorrando así memoria.
* El **modo de etiquetado (label_mode)**, esto es, la codificación de las etiquetas a utilizar. `categorical` significa que utilizaremos la ya conocida codificación one-hot.

Un objeto `Dataset` funciona de manera muy similar a un generador de Python, lo que significa que podemos iterar sobre él para obtener batches de imágenes ya preprocesadas. Por ejemplo, el siguiente código inicia un bucle para extraer todos los batches del `Dataset`, nos muestra el contenido el primero, y detiene la iteración.

In [10]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

Shape of input batch: (64, 128, 128, 3)
Shape of output batch: (64, 6)
Input batch:
[[[[2.60000000e+01 1.15000000e+01 8.50000000e+00]
   [2.57500000e+01 1.12500000e+01 6.00000000e+00]
   [7.17500000e+01 5.65000000e+01 4.47500000e+01]
   ...
   [1.02250000e+02 6.32500000e+01 3.12500000e+01]
   [1.03250000e+02 6.50000000e+01 2.97500000e+01]
   [1.06250000e+02 6.82500000e+01 3.15000000e+01]]

  [[2.52500000e+01 1.32500000e+01 1.20000000e+01]
   [2.52500000e+01 1.20000000e+01 1.07500000e+01]
   [2.67500000e+01 1.35000000e+01 5.00000000e+00]
   ...
   [1.00750000e+02 6.20000000e+01 2.82500000e+01]
   [1.04250000e+02 6.62500000e+01 2.97500000e+01]
   [1.05250000e+02 6.72500000e+01 2.92500000e+01]]

  [[2.62500000e+01 1.25000000e+01 1.10000000e+01]
   [2.77500000e+01 1.27500000e+01 1.25000000e+01]
   [2.92500000e+01 1.42500000e+01 8.00000000e+00]
   ...
   [9.77500000e+01 6.12500000e+01 3.07500000e+01]
   [1.04750000e+02 6.87500000e+01 3.37500000e+01]
   [1.04000000e+02 6.85000000e+01 3.15000

Podemos ver que, efectivamente, el generador produce un tensor de datos de entrada de las dimensiones apropiadas para poder introducirlo en la red neuronal, y que las salidas también se han codificado correctamente como one-hot.
No obstante, todavía hay un problema con los datos: los valores de los píxeles están en el rango [0, 255], lo cual puede producir problemas de entrenamiento. Resolveremos este punto después, en la definición de la red neuronal, mediante una capa especial. Por ahora vamos a cotinuar, definiendo una función que construya los `Dataset` para los datos de entrenamiento, validación y test.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea una función <b>create_datasets</b> que reciba los siguiente parámetros:
    <ul>
      <li><b>traindir</b>: el directorio donde están localizadas las imágenes de entrenamiento.</li>
      <li><b>valdir</b>: el directorio donde están localizadas las imágenes de validación.</li>
      <li><b>testdir</b>: el directorio donde están localizadas las imágenes de test.</li>
      <li><b>image_size</b>: el tamaño que se utilizará para redimensionar todas las imágenes a una resolución común.</li>
      <li><b>batch_size</b>: el tamaño de los batches de imágenes que serán generados.</li>
    </ul>
    La función debe crear objetos `Dataset` para los directorios de entrenamiento, validación y test, y devolver los tres datasets creados como

    return train_dataset, val_dataset, test_dataset
</font>

***

In [43]:
####### INSERT YOUR CODE HERE


def create_datasets(traindir, valdir, testdir, image_size, batch_size):
    if isinstance(image_size, int):
        image_size = (image_size, image_size)

    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        traindir,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical'
    )

    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        valdir,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical'
    )

    test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        testdir,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical'
    )

    return train_dataset, val_dataset, test_dataset

Probemos que la función que has implementado funciona correctamente

In [44]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=64) 

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Convolution2D, Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

¡Ahora que tenemos nuestros `Dataset` podemos entrenar una red profunda con ellos! Como ejemplo, vamos a construir una red convolucional extremadamente simple. Nótese cómo hemos añadido una capa especial de preprocesado llamada `Rescaling`, que será la encargada de normalizar los valores de los píxeles al rango [0, 1] cada vez que la red reciba una imagen.

¡Ojo! Esta red tan simple no producirá errores al ejecutar, pero tiene algunos fallos de diseño que deberás corregir cuando crees tu propia red, más adelante en esta práctica.

El método `fit` de un modelo Keras puede recibir un objeto `Dataset` como datos de entrenamiento, en lugar de un par de tensores (entradas, salidas). Como al construir los `Dataset` ya especificamos el tamaño de batch, no es necesario indicarlo ahora.

In [ ]:
model.fit(train_dataset, epochs=1)

96/96 [==============================] - 6s 55ms/step - loss: nan - accuracy: 0.1664       


Análogamente, podemos evaluar el rendimiento de nuestro modelo sobre el `Dataset` de test de la siguiente manera

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 2s 38ms/step - loss: nan - accuracy: 0.1778
Loss nan, accuracy 17.8%


Este nivel de acierto puede parecer pobre, pero ten en cuenta que hemos usado un modelo muy simple y que el problema es de 6 clases. ¿Serás capaz de hacerlo mejor?

## Construyendo tu propia red

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Diseña una red neuronal profunda que obtenga el mejor acierto posible sobre los datos de test. Puedes usar los datos de entrenamiento y validación como te parezca mejor, pero <b>sólo</b> puedes usar los datos de test para evaluar el acierto de tu modelo. Debes obtener una red capaz de alcanzar al menos un 45% de acierto sobre los datos de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunas recomendaciones y estrategias que pueden ayudar a mejorar tu diseño de red:
    
- <b>Arquitectura</b>: usa todos los trucos que has aprendido en los ejercicios anteriores: capas convolucionales + pooling, activaciones ReLU, dropout... asegúrate también de utilizar un buen optimizador, con una función de error (loss) adecuada, así como una función de activación en la capa de salida que sea adecuada para esta clase de problema (clasificación multiclase).
- <b>Desarrollo incremental</b>: empieza por redes pequeñas, con un número pequeño de parámetros, de forma que puedas comprobar rápidamente qué tal funcionan. Después, puedes hacer tu red más grande en tres direcciones: mayor tamaño de imágenes de entrada, más capas, y más kernels por capa convolucional o unidades por capa densa. Si aumentas el tamaño de las imágenes de entrada, asegúrate de añadir también más capas Convolution+Pooling, para que así a la capa Flatten solo lleguen imágenes muy pequeñas (10x10 píxeles o menos).
- <b>Tamaño de imágenes</b>: configurar los `Dataset` para que carguen imágenes de mayor tamaño puede mejorar significativamente el rendimiento de tu red. Pero ten cuidado, ¡también puedes encontrarte errores de falta de memoria (CUDA memory error) si cargas imágenes a un tamaño demasiado grande! Para esta práctica, un tamaño mayor a 256 puede ser demasiado grande...
- <b>Controlar el número de épocas</b>: Usa una <a href="https://keras.io/api/callbacks/early_stopping/">**estrategia de EarlyStopping**</a> para monitorizar el loss de los datos de validación, y así detener el entrenamiento cuando tras un número de épocas esa loss no haya decrecido. Configurar la EarlyStopping para restaurar los mejores parámetros encontrados durante la optimización también puede resultarte útil.
- <b>Sobreajuste</b>: si tu red obtiene un accuracy casi perfecto en entrenamiento, puede que estés sufriendo sobreajuste (aunque puede que no...). Prueba a incrementar el nivel de Dropout en las capas Dense para comprobar si así obtienes mejoras en el test.
- <b>Demasiado bueno para ser verdad</b>: si tu red obtiene resultados muy buenos, del orden del 90% o más de acierto en test... sospecha. Es posible que estés mezclando los datos de entrenamiento y test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Como ejercicio avanzado, añade las siguiente estrategias a tu red:

- Usa **técnicas de "image augmentation"** para aumentar artificialmente tu dataset de entrenamiento. Para ello, explora las <a href="https://keras.io/api/layers/preprocessing_layers/image_augmentation/">capas de augmentation disponibles en Keras</a>.
- Usa capas de <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> para facilitar el entrenamiento de la red. Revisa en las diapositivas de clase cuál es la forma adecuada de colocarlas en la red.
    
Usando estos trucos y los mencionados en el punto anterior, es posible obtener más de un 60% de acierto en el conjunto de test.

</font>

***

MODELO 1

In [27]:
####### INSERT YOUR CODE HERE

from tensorflow.keras.layers import  MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import  MaxPooling2D
from tensorflow.keras.layers import  BatchNormalization
from tensorflow.keras.layers import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom
from tensorflow.keras.layers import  BatchNormalization


model1 = Sequential()


model1.add(Rescaling(scale=1./255, input_shape=(128, 128, 3)))

model1.add(Convolution2D(64, (3, 3), input_shape=(image_size, image_size, 3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=2, strides=2))

model1.add(Convolution2D(32, (5, 5),  input_shape=(image_size, image_size, 3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))

model1.add(Convolution2D(32, (5, 5),  input_shape=(image_size, image_size, 3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))

model1.add(Convolution2D(32, (5, 5),  input_shape=(image_size, image_size, 3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=2, strides=2))

model1.add(Flatten())

model1.add(Dense(2042, activation="relu"))
model1.add(Dropout(0.6))


model1.add(Dense(6, activation="softmax"))

early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [99]:
model1.fit(train_dataset, 
           batch_size=128, 
           epochs=30, 
           validation_data=val_dataset, 
           callbacks=[early_stopping])

Epoch 1/30
96/96 [==============================] - 19s 152ms/step - loss: 11.0783 - accuracy: 0.2695 - val_loss: 1.8359 - val_accuracy: 0.1822
Epoch 2/30
96/96 [==============================] - 14s 143ms/step - loss: 1.5893 - accuracy: 0.3287 - val_loss: 1.9105 - val_accuracy: 0.1736
Epoch 3/30
96/96 [==============================] - 16s 157ms/step - loss: 1.5324 - accuracy: 0.3454 - val_loss: 1.8865 - val_accuracy: 0.2049
Epoch 4/30
96/96 [==============================] - 14s 140ms/step - loss: 1.5136 - accuracy: 0.3578 - val_loss: 1.8776 - val_accuracy: 0.1689
Epoch 5/30
96/96 [==============================] - 15s 146ms/step - loss: 1.4800 - accuracy: 0.3842 - val_loss: 1.6790 - val_accuracy: 0.2766
Epoch 6/30
96/96 [==============================] - 14s 140ms/step - loss: 1.4619 - accuracy: 0.4012 - val_loss: 1.6335 - val_accuracy: 0.3031
Epoch 7/30
96/96 [==============================] - 15s 145ms/step - loss: 1.5055 - accuracy: 0.3757 - val_loss: 1.4414 - val_accuracy: 0.404

In [100]:
loss, acc = model1.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 6s 150ms/step - loss: 1.3238 - accuracy: 0.4792
Loss 1.32, accuracy 47.9%


<H2> Accuracy en el conjunto de test usando un modelo simple: 47.9% <h2>

MODELO 2  


In [28]:

from tensorflow.keras.layers import  MaxPooling2D
from tensorflow.keras.layers import  BatchNormalization

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomCrop, RandomTranslation
from tensorflow.keras.layers import RandomRotation, RandomZoom

model2 = Sequential()


#model2.add(RandomFlip(mode="horizontal"))
#model2.add(RandomZoom(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3)))
#model2.add(RandomRotation(factor=(-0.3, 0.3)))
#model2.add(RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3)))
#model2.add(RandomCrop(height=32, width=32))

model2.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

model2.add(Convolution2D(256, (3, 3), input_shape=(image_size, image_size, 3)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Convolution2D(128,(3, 3)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Convolution2D(64, (3, 3)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Flatten())

model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(6, activation='softmax'))

early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)


model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])


In [96]:
model2.fit(train_dataset, 
           batch_size=batch_size, 
           epochs=50, 
           validation_data=val_dataset, 
           callbacks=[early_stopping])



Epoch 1/50
96/96 [==============================] - 34s 269ms/step - loss: 2.1608 - accuracy: 0.2552 - val_loss: 1.7651 - val_accuracy: 0.2391
Epoch 2/50
96/96 [==============================] - 25s 250ms/step - loss: 1.7122 - accuracy: 0.2711 - val_loss: 1.7483 - val_accuracy: 0.2372
Epoch 3/50
96/96 [==============================] - 25s 250ms/step - loss: 1.7136 - accuracy: 0.2604 - val_loss: 1.7418 - val_accuracy: 0.2372
Epoch 4/50
96/96 [==============================] - 25s 250ms/step - loss: 1.7197 - accuracy: 0.2601 - val_loss: 1.7386 - val_accuracy: 0.2372
Epoch 5/50
96/96 [==============================] - 25s 249ms/step - loss: 1.6954 - accuracy: 0.2657 - val_loss: 1.7382 - val_accuracy: 0.2362
Epoch 6/50
96/96 [==============================] - 25s 251ms/step - loss: 1.6958 - accuracy: 0.2650 - val_loss: 1.7178 - val_accuracy: 0.2467
Epoch 7/50
96/96 [==============================] - 25s 252ms/step - loss: 1.6744 - accuracy: 0.2766 - val_loss: 1.6627 - val_accuracy: 0.2742

ValueError: ignored

In [97]:
loss, acc = model2.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}") 


33/33 [==============================] - 5s 106ms/step - loss: 1.6075 - accuracy: 0.2981
Loss 1.61, accuracy 29.8%


## Transfer learning

Aunque diseñar nuestra propia red puede producir algunos resultados aceptables, suele ser mejor aprovechar el conocimiento ya existente en una red pre-entrenada. Esto no solo nos lleva a resultados mejores, sino que además nos ahorra mucho tiempo de diseño de la red. Para ello, el módulo [Keras Applications](https://keras.io/api/applications/) contiene varios diseños de redes listos para su uso. Por ejemplo, para hacer uso de la famosa red VGG16 hacemos lo siguiente

In [15]:
from tensorflow.keras.applications import VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

58889256/58889256 [==============================] - 0s 0us/step


Por defecto, todas las redes de Keras Applications están precargadas con los pesos que se obtuvieron al entrenar la red sobre el dataset de la competición de [ImageNet](http://www.image-net.org/). Para adaptar la red a nuestro problema, hemos necesitado especificar la resolución de nuestras imágenes (`input_shape`), así como eliminar las capas de salida (`top`) de la red original, dado que nosotros tendremos un número diferente de clases.

Ahora, ¿cómo hacemos para transferir el aprendizaje de esta red? Vamos a ver cómo implementar la estrategia de "bottleneck features". En primer lugar, marcamos el modelo VGG16 como no entrenable, para que sus parámetros se mantengan congelados.

In [16]:
vgg16_model.trainable = False

Hecho esto, vamos a construir una red neuronal que incluya la VGG16 como una de sus "capas". Es necesario tener en cuenta que la red VGG16 se entrenó realizando una normalización muy específica de las imágenes de entrenamiento, y nosotros debemos seguir ese mismo proceso para que la red se comporte correctamente. Convenientemente, Keras también nos da una funcionalidad para replicar la normalización que la VGG16 necesita.

In [21]:
import tensorflow as tf

from tensorflow.keras.applications.vgg16 import preprocess_input

Podemos probar esta normalización con alguna de las imágenes de nuestro dataset

In [18]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

Before normalizing: [[[241. 255. 251.]
  [241. 255. 251.]
  [241. 255. 251.]]

 [[241. 255. 251.]
  [241. 255. 251.]
  [241. 255. 251.]]

 [[241. 255. 251.]
  [241. 255. 251.]
  [241. 255. 251.]]]
After normalizing: [[[147.061   138.22101 117.32   ]
  [147.061   138.22101 117.32   ]
  [147.061   138.22101 117.32   ]]

 [[147.061   138.22101 117.32   ]
  [147.061   138.22101 117.32   ]
  [147.061   138.22101 117.32   ]]

 [[147.061   138.22101 117.32   ]
  [147.061   138.22101 117.32   ]
  [147.061   138.22101 117.32   ]]]


La normalización realizada por la VGG16 consiste en invertir el orden de los canales de color (RGB -> BGR), y restar los valores medios sobre todo el dataset ImageNet para cada canal de color por separado. Afortunadamente, la función `preprocess_input` que hemos importado hace todo este trabajo por nosotros. Además, podemos incrustar esta función como la primera capa de nuestra red, cumpliendo el papel de la capa `Rescaling` que utilizamos en el apartado anterior. Esto es posible gracias a la capa `Lambda`, que permite construir una capa Keras en base a cualquier función de Tensorflow. De modo que, vamos a comenzar nuestro diseño de red con esta capa.

In [31]:
from tensorflow.keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Tras esto, podemos añadir toda la red VGG16 como si fuera una nueva capa, y nuestras propias capas después de ella. A continuación tenemos un ejemplo de esta clase de diseño, aunque es importante destacar que es un diseño muy sencillo que contiene algunos fallos; una red real para hacer transfer learning debería tener un diseño mejor.

In [ ]:
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 32, 32, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_7 (Flatten)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


Observa cómo en el resumen del modelo podemos ver que la red completa tiene millones de parámetros, pero dado que hemos congelado toda la parte de la red perteneciente a la VGG16, solo unos pocos miles de parámetros son entrenables (trainable): aquellos correspondientes a la capa Dense que hemos colocado al final.

Ya podemos compilar y entrenar el modelo a la manera habitual.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 9s 246ms/step - loss: 226.0015 - accuracy: 0.1865
Loss 2.26e+02, accuracy 18.6%


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Usando la estrategia "bottleneck" presentada, implementa una red que haga transfer learning desde la red VGG16, con un diseño correcto. Si lo haces adecuadamente, esta red debe obtener mejores resultados que con la red que diseñaste en el apartado anterior, y con al menos un 80% de acierto sobre el conjunto de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunos consejos para mejorar tu diseño de red:
    
- Incluye una o más capas Dense, con sus funciones de activación apropiadas, antes de la capa de salida.
- Intenta usar una capa de tipo [GlobalAveragePooling](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) en lugar de la capa Flatten. Esta capa calcula una media de todos los valores de píxeles para cada canal, y en algunas ocasiones produce mejores resultados que la capa Flatten.
- ¡Y no olvides todos los consejos del apartado anterior! También aplican aquí.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>

Para mejorar aún más los resultados de tu red, utiliza las siguientes ideas:

- Usa las estrategias PRO del ejercicio anterio.
- Prueba otras redes pre-entrenadas de <a href="https://keras.io/api/applications/">Keras Applications</a>, como ResNet, Xception o EfficientNet.
- Usa una estrategia de transfer learning más avanzada, como fine-tuning o una combinación de bottleneck features y fine-tuning. Revisa las diapositivas de clase para saber cómo.
   
Si empleas todos estos trucos, es posible alcanzar más de un 90% de acierto en el conjunto de test.

</font>

***

In [29]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [25]:
model_pro = Sequential()
model_pro.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

In [26]:
####### INSERT YOUR CODE HERE

model_pro.add(vgg16_model)


#model_pro.add(Flatten())
model_pro.add(GlobalAveragePooling2D())

model_pro.add(Dense(4096, activation="relu"))
model_pro.add(Dropout(0.5))


model_pro.add(Dense(2048, activation="relu"))
model_pro.add(Dropout(0.5))



model_pro.add(Dense(6, activation="softmax"))

early_stopping = EarlyStopping(patience=7, monitor='val_loss', restore_best_weights=True)
model_pro.summary()



optimazer = Adam(learning_rate=0.0001)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_2 (Lambda)           (None, 128, 128, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_6 (Dense)             (None, 4096)              2101248   
                                                                 
 dropout_3 (Dropout)         (None, 4096)              0         
                                                                 
 dense_7 (Dense)             (None, 2048)              8390656   
                                                      

In [161]:

model_pro.compile(loss='categorical_crossentropy', optimizer=optimazer, metrics=["accuracy"])
model_pro.fit(train_dataset, 
           batch_size=64, 
           epochs=100, 
           validation_data=val_dataset, 
           callbacks=[early_stopping])

Epoch 1/100
96/96 [==============================] - 17s 148ms/step - loss: 3.0728 - accuracy: 0.5197 - val_loss: 0.9705 - val_accuracy: 0.7201
Epoch 2/100
96/96 [==============================] - 15s 150ms/step - loss: 1.4905 - accuracy: 0.6432 - val_loss: 0.7696 - val_accuracy: 0.7453
Epoch 3/100
96/96 [==============================] - 14s 142ms/step - loss: 1.0990 - accuracy: 0.6835 - val_loss: 0.7547 - val_accuracy: 0.7381
Epoch 4/100
96/96 [==============================] - 15s 147ms/step - loss: 0.9160 - accuracy: 0.7156 - val_loss: 0.7071 - val_accuracy: 0.7547
Epoch 5/100
96/96 [==============================] - 14s 144ms/step - loss: 0.7761 - accuracy: 0.7371 - val_loss: 0.7323 - val_accuracy: 0.7396
Epoch 6/100
96/96 [==============================] - 14s 139ms/step - loss: 0.6811 - accuracy: 0.7632 - val_loss: 0.6863 - val_accuracy: 0.7680
Epoch 7/100
96/96 [==============================] - 14s 144ms/step - loss: 0.6356 - accuracy: 0.7798 - val_loss: 0.6731 - val_accuracy:

In [162]:
loss, acc = model_pro.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 95ms/step - loss: 0.6098 - accuracy: 0.7947
Loss 0.61, accuracy 79.5%


In [35]:
from tensorflow.keras.applications import EfficientNetB7

Efficient_model = EfficientNetB7(include_top=False, input_shape=(image_size, image_size, 3))

In [36]:
Efficient_model.trainable = False

In [37]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input




In [38]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

Before normalizing: [[[38.5  25.   15.  ]
  [41.5  24.   10.5 ]
  [58.75 34.25 16.75]]

 [[38.5  25.25 16.5 ]
  [42.5  25.5  12.  ]
  [61.75 39.75 22.25]]

 [[41.75 30.25 21.5 ]
  [40.75 25.75 13.75]
  [52.   31.   15.5 ]]]
After normalizing: [[[38.5  25.   15.  ]
  [41.5  24.   10.5 ]
  [58.75 34.25 16.75]]

 [[38.5  25.25 16.5 ]
  [42.5  25.5  12.  ]
  [61.75 39.75 22.25]]

 [[41.75 30.25 21.5 ]
  [40.75 25.75 13.75]
  [52.   31.   15.5 ]]]


In [47]:
model_pro2 = Sequential()
model_pro2.add(Lambda(preprocess_input, input_shape=(128, 128, 3)))

In [48]:
#model_pro.add(Flatten())

model_pro2.add(Efficient_model)

model_pro2.add(GlobalAveragePooling2D())




model_pro2.add(Dense(2048, activation="relu"))
model_pro2.add(Dropout(0.5))


model_pro2.add(Dense(1024, activation="relu"))
model_pro2.add(Dropout(0.5))


model_pro2.add(Dense(512, activation="relu"))
model_pro2.add(Dropout(0.5))

#model_pro2.add(Dense(32, activation="relu"))
#model_pro2.add(Dropout(0.5))




model_pro2.add(Dense(6, activation="softmax"))

early_stopping = EarlyStopping(patience=7, monitor='val_loss', restore_best_weights=True)
model_pro2.summary()



optimazer = Adam(learning_rate=0.0001)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_3 (Lambda)           (None, 128, 128, 3)       0         
                                                                 
 efficientnetb7 (Functional)  (None, 4, 4, 2560)       64097687  
                                                                 
 global_average_pooling2d_1   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_10 (Dense)            (None, 2048)              5244928   
                                                                 
 dropout_6 (Dropout)         (None, 2048)              0         
                                                                 
 dense_11 (Dense)            (None, 1024)              2098176   
                                                      

In [49]:
model_pro2.compile(loss='categorical_crossentropy', optimizer=optimazer, metrics=["accuracy"])
model_pro2.fit(train_dataset, 
           batch_size=64, 
           epochs=150, 
           validation_data=val_dataset, 
           callbacks=[early_stopping])

Epoch 1/150
96/96 [==============================] - 60s 382ms/step - loss: 1.3985 - accuracy: 0.4622 - val_loss: 0.9218 - val_accuracy: 0.6494
Epoch 2/150
96/96 [==============================] - 32s 323ms/step - loss: 0.9701 - accuracy: 0.6463 - val_loss: 0.7698 - val_accuracy: 0.7173
Epoch 3/150
96/96 [==============================] - 32s 331ms/step - loss: 0.8482 - accuracy: 0.6896 - val_loss: 0.7129 - val_accuracy: 0.7391
Epoch 4/150
96/96 [==============================] - 32s 328ms/step - loss: 0.7494 - accuracy: 0.7330 - val_loss: 0.6986 - val_accuracy: 0.7410
Epoch 5/150
96/96 [==============================] - 32s 328ms/step - loss: 0.7026 - accuracy: 0.7489 - val_loss: 0.6728 - val_accuracy: 0.7562
Epoch 6/150
96/96 [==============================] - 32s 327ms/step - loss: 0.6452 - accuracy: 0.7641 - val_loss: 0.6546 - val_accuracy: 0.7581
Epoch 7/150
96/96 [==============================] - 32s 331ms/step - loss: 0.6165 - accuracy: 0.7780 - val_loss: 0.6310 - val_accuracy:

In [50]:
loss, acc = model_pro2.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 9s 231ms/step - loss: 0.5702 - accuracy: 0.8217
Loss 0.57, accuracy 82.2%


## Informe final y resumen de resultados

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Escribe en la siguiente celda un pequeño informe con:
    <ul>
        <li>Una tabla de resultados, indicando qué diseños de red has probado y qué resultados en test has obtenido. Puede usar un estilo de tabla como el que se muestra abajo.</li>
        <li>De las estrategias y diseños que has ido probando, ¿qué ha funcionado y qué no?</li>
        <li>¿Qué has aprendido con esta práctica?
    </ul>
</font>

***

Ejemplo para la tabla de resultados

|Modelo|Procesado de imágenes|Diseño de red neuronal|Estrategia de entrenamiento|Acierto en test|
|-------|---------------------|----------------------|---------------------------|---------------|
|1|Tamaño 128, batch size 64|Convolutional(4= 1x 64; 3x 32) + Flatten + Dense(2064)|Entrenamiento desde 0|47,9%|
|2|Tamaño 128, batch size 64|I.Augmentation + Convolutional(256) + Convolutional(126) + Convolutional(64) Flatten + Dense(2042)+ dropout(0,5)|Entrenamiento Bottleneck 0|28%|
|pro|Tamaño 128, batch size 64|VGG16 + Global avergage pooling + Dense(4098) + Dense(2048)|Bottleneck features|79.5%|
|pro2|Tamaño 128, batch size 64| Efficient_model + Global avergage pooling + Dense(2048) + Dense(1024) +  Dense(512) |Bottleneck features|82.13%|